# Validation

In this example, we demonstrate how to validate input data for a project.

First, let's import the necessary packages:

In [56]:
from probabilistic_library import ReliabilityProject, SensitivityProject, UncertaintyProject 
from probabilistic_library import DistributionType, ReliabilityMethod, SensitivityMethod, UncertaintyMethod
from utils.models import bligh

Now, let's prepare some projects with both valid and invalid input data:

In [57]:
def valid_reliability_analysis():

    project = ReliabilityProject()
    project.model = bligh

    project.variables["m"].distribution = DistributionType.log_normal
    project.variables["m"].mean = 1.76
    project.variables['m'].scale = 0.1

    project.variables["L"].distribution = DistributionType.normal
    project.variables["L"].mean = 50
    project.variables["L"].deviation = 2.5

    project.variables["c_creep"].distribution = DistributionType.deterministic
    project.variables["c_creep"].mean = 18

    project.variables["delta_H"].distribution = DistributionType.gumbel
    project.variables["delta_H"].shift = 0.53
    project.variables["delta_H"].scale = 0.406

    project.settings.reliability_method = ReliabilityMethod.form
    project.settings.relaxation_factor = 0.75
    project.settings.maximum_iterations = 50
    project.settings.epsilon_beta = 0.01

    return project

In [58]:
def invalid_reliability_analysis():

    project = ReliabilityProject()
    project.model = bligh

    project.variables["m"].distribution = DistributionType.log_normal
    project.variables["m"].mean = 1.76
    project.variables['m'].scale = -0.1 # wrong: scale must be positive

    project.variables["L"].distribution = DistributionType.normal
    project.variables["L"].mean = 50
    project.variables["L"].deviation = 2.5

    project.variables["c_creep"].distribution = DistributionType.uniform
    project.variables["c_creep"].minimum = 18
    project.variables["c_creep"].maximum = 17

    project.variables["delta_H"].distribution = DistributionType.triangular
    project.variables["delta_H"].minimum = 0.0
    project.variables["delta_H"].maximum = -1.0 # wrong: maximum must be greater than minimum
    project.variables["delta_H"].shift = 2.0

    project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
    project.settings.minimum_samples = 1000
    project.settings.maximum_samples = 500 # wrong: maximum_samples must be greater than minimum_samples

    return project

In [59]:
def invalid_sensitivity_analysis():

    project = SensitivityProject()
    project.model = bligh

    project.variables["m"].distribution = DistributionType.log_normal
    project.variables["m"].mean = 1.76
    project.variables['m'].scale = -0.1 # wrong: scale must be positive

    project.variables["L"].distribution = DistributionType.normal
    project.variables["L"].mean = 50
    project.variables["L"].deviation = 2.5

    project.variables["c_creep"].distribution = DistributionType.uniform
    project.variables["c_creep"].minimum = 18
    project.variables["c_creep"].maximum = 17 # wrong: maximum must be greater than minimum

    project.variables["delta_H"].distribution = DistributionType.triangular
    project.variables["delta_H"].minimum = 0.0 
    project.variables["delta_H"].maximum = -1.0 # wrong: maximum must be greater than minimum
    project.variables["delta_H"].shift = 2.0 

    project.settings.sensitivity_method = SensitivityMethod.single_variation
    project.settings.low_value = -0.99 # wrong: value must be between 0 and 1
    project.settings.high_value = 2.99 # wrong: value must be between 0 and 1

    return project

In [60]:
def invalid_uncertainty_analysis():

    project = UncertaintyProject()
    project.model = bligh

    project.variables["m"].distribution = DistributionType.log_normal
    project.variables["m"].mean = 1.76
    project.variables['m'].scale = -0.1 # wrong: scale must be positive

    project.variables["L"].distribution = DistributionType.normal
    project.variables["L"].mean = 50
    project.variables["L"].deviation = 2.5

    project.variables["c_creep"].distribution = DistributionType.uniform
    project.variables["c_creep"].minimum = 18
    project.variables["c_creep"].maximum = 17 # wrong: maximum must be greater than minimum

    project.variables["delta_H"].distribution = DistributionType.triangular
    project.variables["delta_H"].minimum = 0.0 
    project.variables["delta_H"].maximum = -1.0 # wrong: maximum must be greater than minimum
    project.variables["delta_H"].shift = 2.0 

    project.settings.uncertainty_method = UncertaintyMethod.crude_monte_carlo
    project.settings.minimum_samples = -99 # wrong: minimum_samples must be positive (it is not used in Uncertainty analysis)
    project.settings.maximum_samples = -99 # wrong: maximum_samples must be positive

    return project

To validate a project, use the `validate()` method. If the project contains invalid inputs, error messages are displayed.

In [61]:
project_valid = valid_reliability_analysis()
project_valid.validate()

ok


In [62]:
project_invalid = invalid_reliability_analysis()
project_invalid.validate()

Error: m => scale value -0.1 is less than 0.
Error: c_creep => maximum value 17 is less than 18.
Error: delta_H => shift value 2 is greater than -1.
Error: maximum samples value 500 is less than 1000.


In [63]:
project_invalid = invalid_sensitivity_analysis()
project_invalid.validate()

Error: m => scale value -0.1 is less than 0.
Error: c_creep => maximum value 17 is less than 18.
Error: delta_H => shift value 2 is greater than -1.
Error: low value value -0.99 is not greater than 0.
Error: high value value 2.99 is not less than 1.


In [64]:
project_invalid = invalid_uncertainty_analysis()
project_invalid.validate()

Error: m => scale value -0.1 is less than 0.
Error: c_creep => maximum value 17 is less than 18.
Error: delta_H => shift value 2 is greater than -1.
Error: maximum samples value -99 is less than 1.
